## x.1 使用torch对张量进行数据处理 

在第一章中我们主要讲解了tensor的创建，在这第三章节中我们主要讲解如何对这些tensor进行数据处理，主要是在Calculus基础上在DL的主要运用，即Automatic Differentiation自动微分的处理。


## x.2 矩阵各种乘法

Pytorch中转置矩阵使用transpose(...)或者permute(...).contiguous()方法，参考`https://blog.csdn.net/qq_43369406/article/details/130004509`

在Pytorch中各种乘法：参考`https://zhuanlan.zhihu.com/p/514053520`

1. 矩阵乘法

矩阵乘法包括向量点积，矩阵矩阵乘法，矩阵向量乘法，矩阵A矩阵B乘法要求B的列数等于A的行数。

其中一维向量和一维向量的**点积**用`torch.dot()`;

**矩阵-向量乘法**用`torch.mv()`;

**矩阵-矩阵乘法**用`torch.mm()`或者`@`;

2. 哈达玛积

哈达玛积要求两个矩阵的行和列相等。哈达玛积是卷积操作中最常见的操作，也即矩阵对应位置的元素相乘。

**哈达玛积**用`*`;

`*`是向量元素中逐个元素相乘;

In [18]:
import torch

x = torch.arange(4, dtype=torch.float32)

In [20]:
a = torch.arange(12).reshape(3, 4)
b = torch.transpose(a, 0, 1)
print(a, b, sep='\n')
# result = torch.dot(a, b) wrong
# print(result)
print("torch.mm() result is \n{}".format(torch.mm(a, b)))

y = 2 * torch.dot(x, x)
print(y, '\n')

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[ 0,  4,  8],
        [ 1,  5,  9],
        [ 2,  6, 10],
        [ 3,  7, 11]])
torch.mm() result is 
tensor([[ 14,  38,  62],
        [ 38, 126, 214],
        [ 62, 214, 366]])
tensor(28.) 



## x.7 automatic differentiation

x.requires_grad_(True)增加追踪梯度信息

y.backward()反向传播，注意只可以对标量进行反向传播。该步骤会更新需要追踪的参数的梯度矩阵

x.grad查看梯度矩阵

x.grad.zero_()  将梯度信息设置为0，不然梯度信息会累积。常见如在下一epoch中累积

In [22]:
import torch

In [23]:
x = torch.arange(4, dtype=torch.float32)
x

tensor([0., 1., 2., 3.])

当将tensor的requires_grad指定为True后，将会增加required_grad关键字进行梯度自动追踪；

在python中的True和False是大写，可以试做常量写法。True == 1; 但是在C语言中就是小写的，如bool flag = false;

In [24]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
print(x.grad)  # The gradient is None by default
x

None


tensor([0., 1., 2., 3.], requires_grad=True)

我们虽然使用requires_grad追踪了梯度，使用了dot进行计算，但是x.grad并没有值，我们首先需要使用y.backward()进行反向传播才能有值

In [25]:
y = 2 * torch.dot(x, x)
print(y, '\n', x.grad)

tensor(28., grad_fn=<MulBackward0>) 
 None


In [26]:
y.backward()    # 相当于对于每个位置都是对应位置的元素的平方，just like y = 2x^2, y' = 4x, 带入x得值
x.grad

tensor([ 0.,  4.,  8., 12.])

在实际计算中，我们每次都要将模型的参数的梯度重新清零在进行计算，否则会引起梯度爆炸，梯度清零使用x.grad.zero_()

In [27]:
# Reset the gradient
x.grad.zero_()  
y = x.sum()
y.backward()
y, x.grad   # 相当于y = x1 + x2 + ... 对每个变量求偏导数

(tensor(6., grad_fn=<SumBackward0>), tensor([1., 1., 1., 1.]))

一个完整的反向传播计算梯度的过程如下，

In [28]:
x.grad.zero_()
z = torch.tensor([4, 5, 6, 7], requires_grad=True, dtype=torch.float32) # the same as # z = torch.nn.Parameter(torch.tensor([4, 5, 6, 7], dtype=torch.float32))
print(x, z, x.grad, z.grad, sep='::\n')
a = torch.dot(x, z)
a.backward()
print(x.grad, z.grad)

tensor([0., 1., 2., 3.], requires_grad=True)::
tensor([4., 5., 6., 7.], requires_grad=True)::
tensor([0., 0., 0., 0.])::
None
tensor([4., 5., 6., 7.]) tensor([0., 1., 2., 3.])


### x.7.1 非标量变量的反向传播

当你对矩阵进行运算时，你最终得到的值必须得是一个标量y，再对这个标量y进行反向传播

在使用反向传播度的时候需要注意一点，你进行梯度反向传播的量如a.backward()中的a必须得是一个标量，标量即torch.Tensor的维度为0的量，不带有中括号。

In [29]:
x.grad.zero_()
y = x * x   # *是向量元素中逐个元素相乘，并不是矩阵乘法，矩阵乘法用torch.mm或者@
y.sum().backward()  # 因为最终得是一个标量才能进行反向传播，所以要用sum()对y进行降维，即y=x1*x1 + x2*x2, 否则就是y = [x1*x1, x2*x2]
# y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x, y, x.grad

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([0., 2., 4., 6.]))

### x.7.2 使用detach()剥离计算图

注意u = y.detach()是指返回的u是一个从计算图中的剥离的常数，而不是y从计算图中剥离出来了

In [33]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x, z, x.grad, u, x.grad == u

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>),
 tensor([0., 1., 4., 9.]),
 tensor([0., 1., 4., 9.]),
 tensor([True, True, True, True]))